Pipeline for Extracting Keywords from the dataset

In [140]:
import pandas as pd
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
import pickle 

In [2]:
nlp = spacy.load("en_core_web_sm")

path = "/Users/ethanvirtudazo/Desktop/DS105_Dataset/6622_jobs.xls"
df_import = pd.read_excel(path)

df_import.shape

(6623, 11)

In [3]:
df_import.head(2)

,Unnamed: 0,title,details,deadline,opport_type,commence_date,contract_type,location,Renumeration,company,links
0,0,Rothschild & Co - Private Equity Long-Term Int...,This London-based 6-month internship is an exc...,2023-04-30,Internship,2023-07-01 00:00:00,Temporary,London,NaN,Rothschild & Co,https://careers.lse.ac.uk//students/jobs/detai...
1,1,2023 HSBC Global Graduate Programme (Hong Kong...,You’re excited about starting your career and ...,2023-01-06,Graduate employment,2023-07-03 00:00:00,Temporary,Hong KongSingapore,NaN,HSBC (HSBC) - Hong Kong,https://careers.lse.ac.uk//students/jobs/detai...


In [115]:
df1 = df_import.iloc[:, 1:5]
df1.head()

df2 = df_import.iloc[:, [7,9,10]]
df2.head()

pdList = [df1,df2]
df = pd.concat(pdList,axis=1)
df.head()

df.shape

types = df.dtypes
print(types)

df_title = df.iloc[:,0]
df_text_both = df.iloc[:, [0,1]]
df_text = df.iloc[:, [1]]

df_text_both.head()


title                  object
details                object
deadline       datetime64[ns]
opport_type            object
location               object
company                object
links                  object
dtype: object


,title,details
0,Rothschild & Co - Private Equity Long-Term Int...,This London-based 6-month internship is an exc...
1,2023 HSBC Global Graduate Programme (Hong Kong...,You’re excited about starting your career and ...
2,2023 HSBC Global Internship Programme (Hong Ko...,You’re excited about starting your career and ...
3,"Graduate Training Scheme, Capital Markets",Graduate Training Scheme – LondonGreySpark Par...
4,6-Months Internship – Sell-side Tech M&A,"At IPTP, we understand software from decades o..."


In [116]:
df_text.head()

,details
0,This London-based 6-month internship is an exc...
1,You’re excited about starting your career and ...
2,You’re excited about starting your career and ...
3,Graduate Training Scheme – LondonGreySpark Par...
4,"At IPTP, we understand software from decades o..."


In [117]:
type(df_text)

pandas.core.frame.DataFrame

In [118]:
len(df_text)

6623

In [7]:
#ar_text = df_text.to_numpy()
#ar_text = ar_text[:101]

df_text_1 = df_text[:101]
len(df_text_1)

101

In [8]:
# Converting text data of the sample data into nlp object called 'document'

#txt_lst = df_text['details'].tolist()

docs_1 = df_text_1['details'].apply(nlp)
len(docs_1)

101

In [9]:
# Viewing specific nlp doc
docs_1[0]

This London-based 6-month internship is an exciting opportunity to intern with Rothschild & Co's European corporate private equity business. Rothschild & Co’s European corporate private equity business is comprised of three strategies: Five Arrows Principal Investments (FAPI), Five Arrows Growth Capital (FAGC), and Five Arrows Long Term Fund (FALT). FAPI manages c. €1.3 billion through its latest fund and is the flagship corporate private equity / buy-out vehicle investing in mid-cap companies, while FAGC manages c. €500 million through growth investments and small-cap buy-outs, and FALT makes investments in larger companies with the option for longer-term holding periods. All three funds are focused on investments within Western Europe (with FALT additionally on North America) and primarily invest in three main verticals: Data & Software, Healthcare, and Technology-enabled Business Services. The respective teams are based in London, Paris and Luxembourg and are comprised of c. 30 inve

In [10]:
#Individual Job View of NER 
# NER = Name Entity Recognition
rend_doc_1 = docs_1[73]

from spacy import displacy
displacy.render(rend_doc_1,style="ent")

Part-of-Speech Filter

In [11]:
filtered_docs_1 = list([[token.text for token in doc if token.pos_ in ['PROPN', 'NOUN','ADJ']] for doc in docs_1])
len(filtered_docs_1)

101

In [12]:
# Viewing first filtered list of tokens
# words that are not proper nouns, nouns, or adjectives are removed from the documents.
filtered_docs_1[0]

['London',
 'month',
 'internship',
 'exciting',
 'opportunity',
 'Rothschild',
 'Co',
 'European',
 'corporate',
 'private',
 'equity',
 'business',
 'Rothschild',
 'Co',
 'European',
 'corporate',
 'private',
 'equity',
 'business',
 'strategies',
 'Arrows',
 'Principal',
 'Investments',
 'FAPI',
 'Arrows',
 'Growth',
 'Capital',
 'FAGC',
 'Arrows',
 'Long',
 'Term',
 'Fund',
 'FALT',
 'FAPI',
 'c.',
 'latest',
 'fund',
 'flagship',
 'corporate',
 'private',
 'equity',
 'vehicle',
 'mid',
 '-',
 'cap',
 'companies',
 'FAGC',
 'c.',
 'growth',
 'investments',
 'small',
 'cap',
 'buy',
 'outs',
 'FALT',
 'investments',
 'larger',
 'companies',
 'option',
 'longer',
 'term',
 'holding',
 'periods',
 'funds',
 'investments',
 'Western',
 'Europe',
 'FALT',
 'North',
 'America',
 'main',
 'verticals',
 'Data',
 'Software',
 'Healthcare',
 'Technology',
 'Business',
 'Services',
 'respective',
 'teams',
 'London',
 'Paris',
 'Luxembourg',
 'c.',
 'investment',
 'professionals',
 'countries

In [13]:
# text_1 for testing
text_1 = filtered_docs_1[0]

In [14]:
# Might not need this after all
# join all tokens for each job into a single string
fil_lst_1 = [' '.join(lst) for lst in filtered_docs_1]
len(fil_lst_1)

101

In [15]:
type(fil_lst_1[0])

str

In [16]:
type(filtered_docs_1[0])

list

Dictionary Filter

Technical/Hard Skills: 

1. IT Skills
* MS Office 
    * PowerPoint
    * Excel 
    * Word
* Pages
* Numbers
2. Financial Modelling (Modeling)
3. Programming
* Java  
* C# (C #)
* C++ 
* SQL 
* NoSQL
* Perl 
* JavaScript 
* HTML 
* CSS
* Python
* Java 

4. Data Platform Navigation and Utilisation:
* FactSet
* Bloomberg

5. Technical Knowledge
* Interest/Knowledge/Understanding in corporate finance/financial markets/finance/financial services

6. Cognitive Ability
* Analytical/Numerical/Quantitative skills/Problem-Solving


Soft Skills
1. Interpersonal 
* Communication
* Presentation
2. Project Management
* Leadership
* Attention to detail
* Work under pressure







In [17]:
# contains the list of strings for a particular job
text = filtered_docs_1[50]

In [18]:
#Filtering for IT Skills: Microsoft
MS_keys = ["Microsoft","MS","MS-Office","Powerpoint","Excel"]

MS_score_cutoff = 75

MS_match = [match for match in [process.extractOne(key, text, scorer = fuzz.token_set_ratio, score_cutoff = MS_score_cutoff) 
for key in MS_keys] if match is not None and match[1] >= MS_score_cutoff]

print(MS_match)

[]


In [19]:
#Filtering for Financial Modelling 
# assumption: occurence of 'modelling' string is in the context of 'financial modelling'
FM_keys = ["modelling"]

FM_score_cutoff = 75

FM_match = [match for match in [process.extractOne(key, text, scorer = fuzz.token_set_ratio, score_cutoff = FM_score_cutoff)
for key in FM_keys] if match is not None and match[1] >= FM_score_cutoff]

print(FM_match)

[('modelling', 100)]


In [20]:
#Filtering for Data Platform Skills: Bloomberg and/or FactSet
DATA_keys = ["Bloomberg","FactSet"]

DATA_score_cutoff = 75

DATA_match = [match for match in [process.extractOne(key, text, scorer = fuzz.token_set_ratio, score_cutoff = DATA_score_cutoff) 
for key in DATA_keys] if match is not None and match[1] >= DATA_score_cutoff]

print(DATA_match)

[]


Filtering for Programming Skills: Python, SQL, Java, JavaScript, C++, C#

In [21]:
# Programming: Python

PY_keys = ["Python"]

PY_score_cutoff = 75

PY_match = [match for match in [process.extractOne(key, text, scorer = fuzz.token_set_ratio, score_cutoff = PY_score_cutoff) 
for key in PY_keys] if match is not None and match[1] >= PY_score_cutoff]
print(PY_match)

[('Python', 100)]


In [22]:
# Programming: Databases

SQL_keys = ["SQL","mySQL"]

SQL_score_cutoff = 75

SQL_match = [match for match in [process.extractOne(key, text, scorer = fuzz.token_set_ratio, score_cutoff = SQL_score_cutoff) 
for key in SQL_keys] if match is not None and match[1] >= SQL_score_cutoff]
print(SQL_match)

[('SQL', 100), ('SQL', 75)]


In [23]:
# Programming: R

R_keys = ["RStudio", "R"]

R_score_cutoff = 75

R_match = [match for match in [process.extractOne(key, text, scorer = fuzz.token_set_ratio, score_cutoff = R_score_cutoff) 
for key in R_keys] if match is not None and match[1] >= R_score_cutoff]
print(R_match)

[('R', 100)]


In [24]:
# Programming: etc.
# assumption for category: these programming languages occur less than Python, R, SQL, etc.

PETC_keys = ["C++","C#","JAVA","JavaScript","CSS","HTML","PERL"]

PETC_score_cutoff = 75

PETC_match = [match for match in [process.extractOne(key, text, scorer = fuzz.token_set_ratio, score_cutoff = PETC_score_cutoff) for key in PETC_keys] if match is not None and match[1] >= PETC_score_cutoff]
print(PETC_match)

[('C', 100), ('C', 100)]


In [25]:
# Creating a function to pass the texts through

def dict_filter_view(input_text):
    MS_match = [match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = MS_score_cutoff) for key in MS_keys] if match is not None and match[1] >= MS_score_cutoff]
    FM_match = [match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = FM_score_cutoff) for key in FM_keys] if match is not None and match[1] >= FM_score_cutoff]
    DATA_match = [match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = DATA_score_cutoff) for key in DATA_keys] if match is not None and match[1] >= DATA_score_cutoff]
    PY_match = [match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = PY_score_cutoff) for key in PY_keys] if match is not None and match[1] >= PY_score_cutoff]
    R_match = [match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = R_score_cutoff) for key in R_keys] if match is not None and match[1] >= R_score_cutoff]
    SQL_match = [match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = SQL_score_cutoff) for key in SQL_keys] if match is not None and match[1] >= SQL_score_cutoff]
    PETC_match = [match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = PETC_score_cutoff) for key in PETC_keys] if match is not None and match[1] >= PETC_score_cutoff]

    return(
    MS_match,
    FM_match,
    DATA_match,
    PY_match,
    R_match,
    SQL_match,
    PETC_match)


In [35]:
def dict_filter_bool(input_text):
    MS = bool([match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = MS_score_cutoff) for key in MS_keys] if match is not None and match[1] >= MS_score_cutoff])
    FM = bool([match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = FM_score_cutoff) for key in FM_keys] if match is not None and match[1] >= FM_score_cutoff])
    DATA = bool([match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = DATA_score_cutoff) for key in DATA_keys] if match is not None and match[1] >= DATA_score_cutoff])
    PY = bool([match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = PY_score_cutoff) for key in PY_keys] if match is not None and match[1] >= PY_score_cutoff])
    R = bool([match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = R_score_cutoff) for key in R_keys] if match is not None and match[1] >= R_score_cutoff])
    SQL = bool([match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = SQL_score_cutoff) for key in SQL_keys] if match is not None and match[1] >= SQL_score_cutoff])
    PETC = bool([match for match in [process.extractOne(key, input_text, scorer = fuzz.token_set_ratio, score_cutoff = PETC_score_cutoff) for key in PETC_keys] if match is not None and match[1] >= PETC_score_cutoff])

    arr = np.array([
    MS, 
    FM, 
    DATA, 
    PY, 
    R, 
    SQL, 
    PETC])
    arr = arr.astype(int)
    return(arr)

In [27]:
filtered = dict_filter_view(text)
filtered

([],
 [('modelling', 100)],
 [],
 [('Python', 100)],
 [('R', 100)],
 [('SQL', 100), ('SQL', 75)],
 [('C', 100), ('C', 100)])

In [28]:
fil_view = dict_filter_bool(text)
fil_view

array([False,  True, False,  True,  True,  True,  True])

In [30]:
fil_series = pd.Series(filtered_docs_1)

In [31]:
len(fil_series)

101

In [36]:
test_result = fil_series.apply(dict_filter_bool)
test_result

0      [1, 0, 0, 0, 0, 0, 1]
1      [0, 0, 0, 0, 0, 0, 0]
2      [0, 0, 0, 0, 0, 0, 0]
3      [1, 0, 0, 0, 0, 0, 1]
4      [1, 0, 0, 0, 0, 0, 0]
               ...          
96     [0, 0, 0, 0, 0, 0, 0]
97     [1, 0, 0, 0, 0, 0, 0]
98     [0, 1, 0, 0, 0, 0, 1]
99     [1, 1, 0, 0, 0, 0, 0]
100    [1, 0, 0, 0, 0, 0, 0]
Length: 101, dtype: object

In [37]:
test_result_view = fil_series.apply(dict_filter_view)
test_result_view

0      ([(Microsoft, 100), (Office, 100), (PowerPoint...
1                           ([], [], [], [], [], [], [])
2                           ([], [], [], [], [], [], [])
3      ([(MS, 100), (MS, 100), (PowerPoint, 100), (Ex...
4      ([(Office, 100), (PowerPoint, 100), (Excel, 10...
                             ...                        
96                          ([], [], [], [], [], [], [])
97     ([(MS, 100), (MS, 100), (Excel, 100)], [], [],...
98     ([], [(modeling, 94)], [], [], [], [], [(Class...
99     ([(MS, 100), (Office, 100)], [(Dealing, 75)], ...
100    ([(Microsoft, 100), (Office, 100), (EXCEL, 100...
Length: 101, dtype: object

In [ ]:
print(test_result_view.values)

[([('Microsoft', 100), ('Office', 100), ('PowerPoint', 100), ('Excel', 100)], [], [], [], [], [], [('c.', 100), ('c.', 100)])
 ([], [], [], [], [], [], []) ([], [], [], [], [], [], [])
 ([('MS', 100), ('MS', 100), ('PowerPoint', 100), ('Excel', 100)], [], [], [], [], [], [('class', 75)])
 ([('Office', 100), ('PowerPoint', 100), ('Excel', 100)], [], [], [], [], [], [])
 ([('Excel', 100)], [('modeling', 94)], [], [], [], [], [])
 ([('Excel', 100)], [('modeling', 94)], [], [], [], [], [])
 ([('Microsoft', 100), ('office', 100), ('PowerPoint', 100), ('Excel', 100)], [('modelling', 100)], [], [], [], [], [])
 ([('office', 100)], [], [], [], [], [], [])
 ([('offices', 75)], [], [], [], [], [], [('Class', 75)])
 ([('Office', 100)], [], [], [], [], [], [])
 ([('offices', 75)], [], [], [], [], [], [])
 ([('offices', 75)], [], [], [], [], [], [])
 ([('office', 100), ('Powerpoint', 100), ('Excel', 100)], [], [], [], [], [], [])
 ([('Microsoft', 100), ('PowerPoint', 100), ('Excel', 100)], [], [], 

In [ ]:
type(test_result_view)

pandas.core.series.Series

In [ ]:
# CREATING DATAFRAME FOR R

# 1 Turning titles 
df_title_r_1 = df_title[0:101]
df_title_r_1 = pd.DataFrame(df_title_r_1)
type(df_title_r_1)

pandas.core.frame.DataFrame

In [ ]:
df_title_r_1.head()

,title
0,Rothschild & Co - Private Equity Long-Term Int...
1,2023 HSBC Global Graduate Programme (Hong Kong...
2,2023 HSBC Global Internship Programme (Hong Ko...
3,"Graduate Training Scheme, Capital Markets"
4,6-Months Internship – Sell-side Tech M&A


In [ ]:
df_fil_1 = pd.DataFrame(fil_lst_1)
type(df_fil_1)

pandas.core.frame.DataFrame

In [ ]:
len(df_fil_1)

101

In [ ]:
# CREATING DATAFRAME FOR R

df_r_1 = pd.concat([df_title_r_1, df_fil_1], axis = 1)
df_r_1

,title,0
0,Rothschild & Co - Private Equity Long-Term Int...,London month internship exciting opportunity R...
1,2023 HSBC Global Graduate Programme (Hong Kong...,excited career many paths possibilities global...
2,2023 HSBC Global Internship Programme (Hong Ko...,excited career many paths possibilities global...
3,"Graduate Training Scheme, Capital Markets",Graduate Training Scheme LondonGreySpark Partn...
4,6-Months Internship – Sell-side Tech M&A,IPTP software decades deep experience technolo...
...,...,...
96,M&A Analyst Intern,MAJOR RESPONSIBILITIESGather financial operat...
97,12 Month Internship - Financial Crimes and San...,Job summaryFinancial Crime Financial Security ...
98,Investment Associate - Fixed Income,Position OverviewPutnam energetic curious indi...
99,12 Month Internship - Central Compliance,SummaryThe Central Compliance team responsible...


In [ ]:
df_r_1.iloc[95,1]

'2R Capital Investment Management Limited independent investment company London UK successful credit business process new initiatives equity investing private assets equity primary objective long term capital clients commensurate reasonable risk attention mid - sized European companies fundamental investors extensive research businesses regions industry sectors significant expertise private assets space equity debt small medium sized companies significant growth potential sectors regions Job Opportunities analysts investment opportunities Europe internship full time positions available Targeted training successful candidates self starters activities little supervision keen interest securities investing good research writing financial modelling abilities European languages important Day day activities search origination potential investment opportunities primary research analysis specific sectors companies valuation investment opportunities Direct interaction entrepreneurs managers inve

In [ ]:
#Export as .csv file
#df_r_1.to_csv('df_r_1.csv')

Below is the code for applying the pipeline to the entire dataset

1. Cleaning the data 
-identify index of non-text data
-removing nan (non-text) data

2. Convert to nlp object 'document'


3. 

Running Code on the Entire Dataset

In [40]:
type(df_text)

pandas.core.frame.DataFrame

In [98]:
# convering the column of texts into a list of text
lst_lst = df_text.values.tolist()
len(lst_lst)

6623

In [100]:
type(lst_lst[0])

list

In [101]:
# storing the data type of every element in the list of texts (which contains all of the job details)
# we expect the text data to be a 'str' object
# non_str_index tells us which indexes include non-string values and therefore should be removed
non_str_index = [i for i, sublst in enumerate(lst_lst) if any(not isinstance(val, str) for val in sublst)]
non_str_index

[1800, 3395, 6081]

In [142]:
len(df_text)

6623

In [143]:
df_text_filtered = df_text.drop(non_str_index, axis=0, inplace=False)
len(df_text_filtered)

6620

In [144]:
df_text_all = df_text_filtered
len(df_text_all)

6620

In [133]:
# use this to apply the nlp() to the entire dataset

#all_docs = df_text_all['details'].apply(nlp)
#len(all_docs)

6620

In [135]:
# use this to store the list of documents locally (running nlp() again takes time)

#with open("all_spacy_docs.pickle", "wb") as f:
#    pickle.dump(all_docs, f)

In [136]:
with open("all_spacy_docs.pickle", "rb") as f:
    loaded_all_docs = pickle.load(f)

In [145]:
len(loaded_all_docs)

6620